In [1]:
import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.notebook import trange, tqdm, tqdm_notebook
from joblib import Parallel, delayed
import re
import h5py
import tifffile as tiff


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()
data_processed = data_dir / 'processed'
data_raw = r'Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\Specificity experiments\Bim IF comparison'


In [4]:
import os
 
os.environ['JDK_HOME'] = "C:\Program Files\Java\jdk-17"
os.environ['JAVA_HOME'] = "C:\Program Files\Java\jdk-17"

os.environ['PATH'] += ';C:\\Program Files\\Java\\jdk-17\\jre\\bin\\server\\;C:\\Program Files\\Java\\jdk-17\\bin\\;C:\\Program Files\\Java\\jdk-17\\bin\\jar.exe'

# Get info

In [9]:
markers_map = {
    'cycle1': {
        1: 'DNA', 
        3: 'Bim',
    },
    'cycle2': {
        1: 'DNA', 
        2: 'Concanavalin A',
        3: 'Phalloidin',
        4: 'WGA',
    },
}

def get_info(data_raw, marker_dict):
    conditions = []
    fovs = []
    cycles = []
    channels = []
    markers = []
    rois = []
    z_stacks = []
    paths = [] 
    
    # Loop through image folder
    for (dirpath, dirnames, filenames) in os.walk(data_raw):
        if 'after nuclease' in dirpath or 'Test' in dirpath or 'wrong' in dirpath:
            continue
        
            
        for name in sorted(filenames):
            if "tif" in name and "sti" not in name \
            and 'over' not in name.lower() \
            and 'comp' not in name.lower():
                # Get information from image name
                c = filenames[-1][-1] 
                
                if c  == '6':
                    condition = 'KO'
                elif c == '8':
                    condition = 'Control'
                
                d_split = dirpath.split('\\')
                well = d_split[-2].split('_')[1].split(' ')[0]
                n_split = name.split('_')
                roi = int(n_split[2])
                z = int(n_split[3][1:])
                ch = int(n_split[4][2])
                
                if 'Seg' in dirpath:
                    cycle = 'cycle2'
                else:
                    cycle = 'cycle1'
                try: marker = marker_dict[cycle][ch] 
                except: continue
                
                conditions.append(condition)
                cycles.append(cycle)
                channels.append(ch)
                markers.append(marker)
                rois.append(roi)
                z_stacks.append(z)
                paths.append(os.path.join(dirpath, name))
                
    info = {
            "Condition": conditions,
            'FOV': conditions, 
            "Cycle": cycles,
            "Channels": channels,
            "Markers": markers,
            "ROI": rois,
            "Z": z_stacks,
            "Path": paths
        }

    df = pd.DataFrame(info)
    return df

In [31]:
df_meta_path = data_dir / 'specificity_IF_best' / 'metadata' / 'info.csv'
df_meta_path.parent.mkdir(parents=True, exist_ok=True)


df_exist = df_meta_path.is_file()

if not df_exist:
    print('Created df')
    df = get_info(data_raw, markers_map)
    df.to_csv(df_meta_path, index=False)
else:
    print('Loaded df')
    df = pd.read_csv(df_meta_path)

Created df


# Save HDF5

In [23]:
import h5py
from natsort import natsorted, natsort_keygen

def save_hdf5(
    path: str, name: str, data: np.ndarray, attr_dict=None, mode: str = "a"
) -> None:
    # Read h5 file
    hf = h5py.File(path, mode)
    # Create z_stack_dataset
    if hf.get(name) is None:
        data_shape = data.shape
        data_type = data.dtype
        max_shape = (data_shape[0],) + data_shape[1:]
        dset = hf.create_dataset(
            name,
            shape=data_shape,
            maxshape=max_shape,
            chunks=True,
            dtype=data_type,
            compression="gzip",
        )
        dset[:] = data
        if attr_dict is not None:
            for attr_key, attr_val in attr_dict.items():
                dset.attrs[attr_key] = attr_val
    else:
        print(f"Dataset {name} exists")

    hf.close()

def read_img(path):
    return skimage.io.imread(path, as_gray=True)

def joblib_loop(task, pics):
    return Parallel(n_jobs=20)(delayed(task)(i) for i in pics)

In [37]:
df_imgs_path = data_dir / 'specificity_IF_best' / 'metadata' / 'imgs.csv'
df_imgs_path.parent.mkdir(parents=True, exist_ok=True)

temp_path = data_dir  /'specificity_IF_best' /  'hdf5' / 'raw'
temp_path.mkdir(parents=True, exist_ok=True)

df_exist = df_imgs_path.is_file()

if not df_exist:
    print('Created df')
    
    group = df.groupby(['Condition','FOV', 'ROI'])
    rows = []
    
    for name, df_group in tqdm(group, total=len(group)):
        df_group = df_group.sort_values( # Sort by cycle and channels
            by=["Cycle", "Channels", 'Z'],
            key=natsort_keygen()
        )
        file_name = '_'.join(np.array(name).astype(str)) + '.hdf5'
        file_path = temp_path / file_name
        rows.append(list(name)+[file_path])
        
        group_channel = df_group.groupby(['Cycle', 'Channels'])
        for n, df_channel in group_channel:
            try:
                if test_data_exist(file_path, '_'.join(np.array(n).astype(str))):
                    continue
            except:pass
            
            marker = df_channel.iloc[0].Markers
            paths = df_channel.Path.to_numpy()

            imgs = joblib_loop(read_img, paths)
            imgs = np.array(imgs)
            info = {"Cycle": n[0], "Channel": n[1], "Marker": marker, "Z": df_channel.Z.to_numpy()}
            
            # hdf5 as Channel -> Z mapping
            save_hdf5(file_path, '_'.join(np.array(n).astype(str)), imgs, info)
    df_imgs = pd.DataFrame(rows, columns=['Condition','FOV', 'ROI', 'Path'])        
    df_imgs.to_csv(df_imgs_path, index=False)
else:
    print('Loaded df')
    df_imgs = pd.read_csv(df_imgs_path)

Created df


  0%|          | 0/18 [00:00<?, ?it/s]

In [38]:
df_imgs

,Condition,FOV,ROI,Path
0,Control,Control,1,Y:\coskun-lab\Thomas\23_PLA_revision\data\spec...
1,Control,Control,2,Y:\coskun-lab\Thomas\23_PLA_revision\data\spec...
2,Control,Control,3,Y:\coskun-lab\Thomas\23_PLA_revision\data\spec...
3,Control,Control,4,Y:\coskun-lab\Thomas\23_PLA_revision\data\spec...
4,Control,Control,5,Y:\coskun-lab\Thomas\23_PLA_revision\data\spec...
5,Control,Control,6,Y:\coskun-lab\Thomas\23_PLA_revision\data\spec...
6,Control,Control,7,Y:\coskun-lab\Thomas\23_PLA_revision\data\spec...
7,Control,Control,8,Y:\coskun-lab\Thomas\23_PLA_revision\data\spec...
8,Control,Control,9,Y:\coskun-lab\Thomas\23_PLA_revision\data\spec...
9,KO,KO,1,Y:\coskun-lab\Thomas\23_PLA_revision\data\spec...


# Calculate best focus

In [22]:
from joblib import Parallel, delayed
import cv2 
import scipy 

# compute laplacian for each z plane in parallel
def computeLapVar(plane):
    
    var = cv2.Laplacian(plane, cv2.CV_64F, ksize = 31)
    var = np.var(var)
    
    return var

# find focus plane via Laplacian variance
def findFocusLapVar(subStack):
    
    lapVar = Parallel(n_jobs = -1, prefer = 'threads', verbose = 0)(delayed(computeLapVar)(subStack[ii, :, :]) for ii in range(subStack.shape[0]))
    idxFocus = np.argmax(lapVar)

    xRange = np.arange(0, len(lapVar))
    
    # compute steepest gradient in variance to find focus plane
    grad = np.gradient(lapVar)
    grad = np.square(grad)
    
    # extract peaks of gradient
    thresh = np.percentile(grad, 50)
    # peaks with min horizontal distance
    peaks, props = scipy.signal.find_peaks(x = grad, height = thresh, distance = 1)
    heights = props['peak_heights']
    
    # idxFocus = np.argmax(grad) + 2
    if len(peaks) == 0:
        idxFocus = len(lapVar) // 2 # middle
        
    else:
        idxFocus = peaks[0] # tallest peak
        
    if idxFocus > len(lapVar) - 2: # exceeds length, out of bounds
        idxFocus = len(lapVar) - 2
        
    return idxFocus, peaks, lapVar

# register cycles across Z by matching focus planes
def registerAlongZ(fixed, moving): # each ZYX
     
    # match moving to fixed
    focusFixed, peaksFixed = findFocusLapVar(fixed) # focus plane of fixed
    focusMoving, peaksMoving = findFocusLapVar(moving) # focus plane of moving
    
    # if multiple peaks found, choose closest to middle 
    if len(peaksFixed) > 1:
        # print('Multiple fixed peaks found...')
        middle = fixed.shape[0] // 2
        diff = np.abs(peaksFixed - middle)
        closest = np.argmin(diff)
        focusFixed = peaksFixed[closest]
        
    if len(peaksMoving) > 1:
        # print('Multiple moving peaks found...')
        middle = moving.shape[0] // 2
        diff = np.abs(peaksMoving - middle)
        closest = np.argmin(diff)
        focusMoving = peaksMoving[closest]
    
    # print('Fixed focus plane is', focusFixed + 1, '/', fixed.shape[0])
    # print('Moving focus plane is', focusMoving + 1, '/', moving.shape[0])
    shift = focusFixed - focusMoving # z shift applied to moving to match fixed
    
    return shift

In [23]:
# row = df_imgs.iloc[1]
# path = row.Path
# print(row)
# with h5py.File(path, "r") as f:
#     for k in tqdm(f.keys(), total=len(f.keys()), leave=False):
#         imgs = f[k][:]
#         break

# plt.imshow(imgs[0])

In [25]:
df_imgs_path = data_dir / 'specificity_IF_best' / 'metadata' / 'imgs.csv'
df_imgs = pd.read_csv(df_imgs_path)

In [39]:
from collections import defaultdict 

best_focus = defaultdict(list)
row = df_imgs.iloc[1]
path = row.Path

with h5py.File(path, "r") as f:
    for k in tqdm(f.keys(), total=len(f.keys()), leave=False):
        cycle = k.split('_')[0]
        channel = f[k].attrs['Channel']

        if channel == 1:
            imgs = f[k][:]

            z, p, lapVar = findFocusLapVar(imgs)
            
            best_focus[cycle] = np.argmax(lapVar)
        break

  0%|          | 0/6 [00:00<?, ?it/s]

In [42]:
# import pickle 
# with open(f'{row.Condition}.pkl', 'wb') as f:
#     pickle.dump(best_focus,f)

# Save Tifffile

In [44]:
import tifffile as tiff
from skimage import exposure, util
import pickle 

save_path = data_dir / 'specificity_IF_best' / 'imgs' / 'raw_norm'

def contrast_str(img, n_min=0.01, n_max=100):
    p2, p98 = np.percentile(img, (n_min, n_max))
    img_rescale = exposure.rescale_intensity(img, in_range=(p2, p98))
    return img_rescale

def make_imgs_same_dim(imgs):
    # Get max dimensions
    shapes = np.array([img.shape[1:] for img in imgs])
    min_x, min_y = shapes.min(axis=0)
    imgs_cropped = [img[:, :min_x, :min_y] for img in imgs]
    for i, img in enumerate(imgs_cropped):
        imgs_cropped[i][0,...] = contrast_str(imgs_cropped[i][0,...])
    return imgs_cropped

In [45]:
save_path.mkdir(parents=True, exist_ok=True)
n = 1
group = df_imgs.groupby(['Condition','FOV'])
for name, df_group in group:
    print('_'.join(name)+'.pkl')
    with open(f'{row.Condition}.pkl', 'rb') as f:
        best_focus = pickle.load(f)
        
    for i, row in tqdm(enumerate(df_group.itertuples()), total=len(df_group)):
        path = row.Path

        # Read images
        cycles = []
        imgs_all = []
        channels = []
        with h5py.File(path, "r") as f:
            for k in tqdm(f.keys(), total=len(f.keys()), leave=False):
                cycle = k.split('_')[0]
                channel = f[k].attrs['Channel']

                imgs = f[k][:]
                cycles.append(cycle)
                channels.append(channel)
                imgs_all.append(imgs)
                       
        cycles = np.array(cycles)
        channels = np.array(channels)

        # Register by z focus
        imgs_matched_z = [] 
        for i, img in enumerate(imgs_all):
            best = best_focus[cycles[i]]
            imgs_matched_z.append(img[best-n:best+n])

        # Get imgs_stacked
        imgs_stacked = np.stack(imgs_matched_z)
        for cycle in np.unique(cycles):
            indices = np.where(cycles == cycle)[0]
            imgs = imgs_stacked[indices,...]
            for z in range(imgs.shape[1]):
                 # Create temp path
                name = [row[1], row[2], z]
                temp_path =  save_path / '_'.join(np.array(name).astype(str))
                temp_path.mkdir(parents=True, exist_ok=True)

                file_name = '_'.join(np.array(["{:03d}".format(row[3]), cycle]).astype(str)) + '.tif'
                file_path = temp_path / file_name

                # if os.path.exists(file_path):
                #     continue
                
                # Write image
                tiff.imwrite(file_path, imgs[:,z,...])

Control_Control.pkl


  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

# Register

In [5]:
from ashlar import fileseries, thumbnail,reg
import matplotlib.pyplot as plt
from ashlar.scripts.ashlar import process_axis_flip

In [6]:
# Loop all images
thumb_dir = data_dir / 'specificity_IF_best' / 'thumbnails'
imgs_dir = data_dir / 'specificity_IF_best'/ 'imgs' / 'raw_norm'
save_dir = data_dir / 'specificity_IF_best'/ 'imgs' / 'registered_norm'

save_dir .mkdir(parents=True, exist_ok=True)
imgs_dir_list = os.listdir(imgs_dir)
thumb_dir.mkdir(parents=True, exist_ok=True)

for dir_path in tqdm(imgs_dir_list):
        
    # Create reader for each cycle
    readers = []
    for i in range(1, 3):
        reader = fileseries.FileSeriesReader(
            str(imgs_dir / dir_path),
            pattern='{series}_cycle'+f'{i}.tif',
            overlap=0.29,
            width=3,
            height=3,
            layout='snake',
            direction='horizontal',
            pixel_size=0.18872, 
        )
        readers.append(reader)
    reader_1 = readers[0]
    
    # Run stitching
    aligner0 = reg.EdgeAligner(reader_1, channel=0, filter_sigma=2, verbose=False,)
    aligner0.run()
    
    # Generate merge image for 1 cycle
    # Parramter
    mosaic_args = {}
    mosaic_args['verbose'] = False

    mosaic = reg.Mosaic(
            aligner0,aligner0.mosaic_shape,**mosaic_args
        )
    writer_class = reg.TiffListWriter
    writer = writer_class(
            [mosaic], str(save_dir / (dir_path + '_cycle1_ch{channel}.ome.tif'))
    )
    writer.run()
    
    # Loop through rest of cycles
    aligners = list()
    aligners.append(aligner0)

    for j in range(1, len(readers)):
        aligners.append(
            reg.LayerAligner(readers[j], aligners[0], channel=0, filter_sigma=2, verbose=False)
        )
        aligners[j].run()
        mosaic = reg.Mosaic(
            aligners[j], aligners[0].mosaic_shape,**mosaic_args
        )
        writer = writer_class(
                [mosaic], str(save_dir / (dir_path +'_cycle'+str(j+1)+'_ch{channel}.ome.tif'))
        )
        writer.run()

  0%|          | 0/8 [00:00<?, ?it/s]

    assembling thumbnail 9/9
    assembling thumbnail 9/9
    estimated cycle offset [y x] = [15. 17.]
    assembling thumbnail 9/9
    assembling thumbnail 9/9
    estimated cycle offset [y x] = [15. 17.]
    assembling thumbnail 9/9
    assembling thumbnail 9/9
    estimated cycle offset [y x] = [12. 16.]
    assembling thumbnail 9/9
    assembling thumbnail 9/9
    estimated cycle offset [y x] = [12. 16.]
    assembling thumbnail 9/9
    assembling thumbnail 9/9
    estimated cycle offset [y x] = [-1. 25.]
    assembling thumbnail 9/9
    assembling thumbnail 9/9
    estimated cycle offset [y x] = [-1. 25.]
    assembling thumbnail 9/9
    assembling thumbnail 9/9
    estimated cycle offset [y x] = [-2. 24.]
    assembling thumbnail 9/9
    assembling thumbnail 9/9
    estimated cycle offset [y x] = [-2. 24.]


# Save data

In [15]:
def get_info(data_raw, marker_dict):
    conditions = []
    fovs = []
    Zs = []
    cycles = []
    channels = []
    markers = []
    paths = [] 

    # Loop through image folder
    for (dirpath, dirnames, filenames) in os.walk(data_raw):
        for name in sorted(filenames):
            if "tif" in name:
                # Get information from image name
                n_split = name.split('_')
                                
                cond=n_split[0]
                fov=n_split[1]
                z=n_split[2]
                cycle=n_split[3]
                ch = n_split[4][:3]
                try:
                    marker = marker_dict[cycle][ch]
                except:
                    continue 
                    
                conditions.append(cond)
                fovs.append(fov)
                Zs.append(z)
                cycles.append(cycle)
                channels.append(ch)
                markers.append(marker)
                paths.append(os.path.join(dirpath, name))
                
    info = {
            "Condition": conditions,
            "FOV": fovs,
            "Z": Zs,
            "Cycle": cycles,
            "Channels": channels,
            "Markers": markers,
            "Path": paths
        }

    df = pd.DataFrame(info)
    return df

def joblib_loop(task, pics):
    return Parallel(n_jobs=20)(delayed(task)(i) for i in pics)

def read_img(path):
    return skimage.io.imread(path, as_gray=True)

def get_min(imgs):
    shapes = np.array([np.array(img.shape) for img in imgs])
    return np.min(shapes, axis=0)

In [16]:
from collections import defaultdict

markers_map_new = defaultdict(dict)
for k,v in markers_map.items():
    for i, (ch,marker) in enumerate(v.items()):
        markers_map_new[k][f'ch{i}'] = marker
        

In [17]:
data_raw = data_dir  / 'specificity_IF_best' /'imgs' / 'registered_norm'
df_meta_path = data_dir / 'specificity_IF_best'  / 'metadata' / 'info_sti.csv'

try:
    df_meta_path.parent.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Folder is already there")

df_exist = df_meta_path.is_file()

if not df_exist:
    print('Created df')
    df = get_info(data_raw, markers_map_new)
    df.to_csv(df_meta_path, index=False)
else:
    print('Loaded df')
    df = pd.read_csv(df_meta_path)

Folder is already there
Loaded df


In [18]:
from natsort import natsorted, natsort_keygen


In [20]:
# Get the min dimension accross all z stacks 
group = df.groupby(['Condition','FOV'])
min_dim = {}

for name, df_group in tqdm(group, total=len(group)):

    df_group = df_group.sort_values( # Sort by cycle and channels
        by=["Cycle", "Channels", 'Z'],
        key=natsort_keygen()
    )
    df_group = df_group[(df_group.Cycle == 'cycle1') & (df_group.Channels == 'ch1')]

    channels = df_group.Channels.to_list()
    cycles = df_group.Cycle.to_list()
    markers = df_group.Markers.to_list()
    paths = df_group.Path.to_numpy()

    imgs = joblib_loop(read_img, paths)
    min_dim['_'.join(name)] = get_min(imgs)

  0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
min_dim

{'Control_Control': array([3473, 4623]), 'KO_KO': array([3470, 4647])}

In [24]:
df_imgs_path = data_dir /'specificity_IF_best'/'metadata' / 'imgs_reg.csv'

temp_path =data_dir / 'specificity_IF_best'/ 'hdf5' / 'registered'
try:
    temp_path.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Folder is already there")

df_exist = df_imgs_path.is_file()

if not df_exist:
    print('Created df')
    group = df.groupby(['Condition','FOV', 'Z'])
    rows = []

    for name, df_group in tqdm(group, total=len(group)):
        file_name = '_'.join(np.array(name).astype(str)) + '.hdf5'
        file_path = temp_path / file_name
        rows.append(list(name)+[file_path])
        
        df_group = df_group.sort_values( # Sort by cycle and channels
            by=["Cycle", "Channels", 'Z'],
            key=natsort_keygen()
        )
        
        if file_path.exists():
            continue
        
        channels = df_group.Channels.to_list()
        cycles = df_group.Cycle.to_list()
        markers = df_group.Markers.to_list()
        paths = df_group.Path.to_numpy()
            
        imgs = joblib_loop(read_img, paths)
        min_shape = min_dim['_'.join(name[:2])]
        imgs_cropped = np.array([img[:min_shape[0], :min_shape[1]] for img in imgs])
        info = {"Cycle": cycles, "Channel": channels, "Marker": markers}
            
            # hdf5 as Channel -> Z mapping
        save_hdf5(file_path, 'imgs', imgs_cropped, info)
    df_imgs = pd.DataFrame(rows, columns=['Condition', 'FOV', 'Z', 'Path'])        
    df_imgs.to_csv(df_imgs_path, index=False)
else:
    print('Loaded df')
    df_imgs = pd.read_csv(df_imgs_path)

Folder is already there
Created df


  0%|          | 0/8 [00:00<?, ?it/s]

# Segmentation

In [5]:
from skimage.exposure import match_histograms, rescale_intensity
import cv2
from skimage import img_as_ubyte
import tensorflow as tf 
from skimage import exposure, util, filters, restoration

def random_crop(image, NEW_IMG_HEIGHT, NEW_IMG_WIDTH):
    cropped_image = tf.image.random_crop(
      image, size=[3, NEW_IMG_HEIGHT, NEW_IMG_WIDTH])

    return cropped_image

def make_color_overlay(input_data):
    """Create a color overlay from 2 channel image data
    
    Args:
        input_data: stack of input images
    
    Returns:
        numpy.array: color-adjusted stack of overlays in RGB mode
    """
    RGB_data = np.zeros(input_data.shape[:3] + (3, ), dtype='float32')
    
    # rescale channels to aid plotting
    for img in range(input_data.shape[0]):
        for channel in range(input_data.shape[-1]):
            # get histogram for non-zero pixels
            percentiles = np.percentile(input_data[img, :, :, channel][input_data[img, :, :, channel] > 0],
                                            [0, 100])
            rescaled_intensity = rescale_intensity(input_data[img, :, :, channel],
                                                       in_range=(percentiles[0], percentiles[1]),
                                                       out_range='float32')
            RGB_data[img, :, :, channel + 1] = rescaled_intensity
        
    # create a blank array for red channel
    return RGB_data

def contrast_str(img, n_min=10, n_max=100):
    p2, p98 = np.percentile(img, (n_min, n_max))
    img_rescale = rescale_intensity(img, in_range=(p2, p98))
    img_rescale = util.img_as_ubyte(img_rescale)
    return img_rescale

In [6]:
df_imgs_path = data_dir /  'specificity_IF_best'  /'metadata' / 'imgs_reg.csv'
df_imgs = pd.read_csv(df_imgs_path)


In [7]:
df_imgs

,Condition,FOV,Z,Path
0,Control,Control,0,Y:\coskun-lab\Thomas\23_PLA_revision\data\spec...
1,Control,Control,1,Y:\coskun-lab\Thomas\23_PLA_revision\data\spec...
2,Control,Control,2,Y:\coskun-lab\Thomas\23_PLA_revision\data\spec...
3,Control,Control,3,Y:\coskun-lab\Thomas\23_PLA_revision\data\spec...
4,KO,KO,0,Y:\coskun-lab\Thomas\23_PLA_revision\data\spec...
5,KO,KO,1,Y:\coskun-lab\Thomas\23_PLA_revision\data\spec...
6,KO,KO,2,Y:\coskun-lab\Thomas\23_PLA_revision\data\spec...
7,KO,KO,3,Y:\coskun-lab\Thomas\23_PLA_revision\data\spec...


In [8]:
for row in tqdm(df_imgs.itertuples(), total=len(df_imgs)):
    path = row.Path
    
    # Read images
    with h5py.File(path, "r") as f:
        
        markers = f['imgs'].attrs['Marker']
        imgs = f['imgs'][:]
    break

  0%|          | 0/8 [00:00<?, ?it/s]

In [9]:
imgs.dtype

dtype('uint16')

In [10]:
import napari 

napari.view_image(np.stack(imgs), name=markers, channel_axis=0, contrast_limits=[0,2**16])

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 1736.0, 2311.0), zoom=0.36189173625107973, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 3473.0, 1.0), (0.0, 4623.0, 1.0)), current_step=(1736, 2311), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'DNA' at 0x1d71fa89bd0>, <Image layer 'Bim' at 0x1d71e1f30a0>, <Image layer 'DNA [1]' at 0x1d72131d870>, <Image layer 'Concanavalin A' at 0x1d7213d8220>, <Image layer 'Phalloidin' at 0x1d725d49bd0>, <Image layer 'WGA' at 0x1d725df02b0>], scale_bar=ScaleBar(visible=False, colored=False, color=array([1., 0., 1., 1.], dtype=float32), ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, box=False, box_color=array([0. , 0. 

In [11]:
df_imgs.sort_values('Z', inplace=True)
df_imgs[['FOV', 'Z']] = df_imgs[['FOV', 'Z']].astype('str')

In [15]:
# Define markers to use for cytosolic segmentation
cyto_markers = ['Phalloidin']

# Define folder and create if don't exsit
whole_seg_path = data_dir /  'specificity_IF_best'  / 'imgs' / 'segmentation'
whole_seg_path.mkdir(parents=True, exist_ok=True)

# Save combined images
for row in df_imgs.itertuples():
    # Read image
    path = row.Path
    with h5py.File(path, "r") as f:
        imgs = f['imgs'][:]
        markers = f['imgs'].attrs['Marker']

    # Get dapi and cyto imgaes
    indices = np.isin(markers, cyto_markers)
    img_dapi =  imgs[-4]
    imgs_cyto = imgs[indices,:]
    
    # Contrast streching and combine to rgb image
    img_dapi = contrast_str(img_dapi, n_max=99.9)
        
    imgs_cyto_scaled = []
    for img in imgs_cyto:
        imgs_cyto_scaled = [contrast_str(img, n_max=99.9)]
    img_cyto = np.max(np.array(imgs_cyto_scaled), axis=0)
    img_rgb = np.stack([np.zeros(img_dapi.shape),img_cyto, img_dapi], axis=0).astype(np.uint8)
    
    # Crop and save
    file_name = f'{"_".join(row[1:4])}.tif'
    file_path = whole_seg_path / file_name
    tiff.imwrite(file_path, img_rgb)



In [14]:
napari.view_image(imgs)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 1734.5, 2323.0), zoom=0.15194524495677234, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(3.0, 1.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=2, last_used=0, range=((0.0, 6.0, 1.0), (0.0, 3470.0, 1.0), (0.0, 4647.0, 1.0)), current_step=(3, 1735, 2323), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'imgs' at 0x1d703c13c10>], scale_bar=ScaleBar(visible=False, colored=False, color=array([1., 0., 1., 1.], dtype=float32), ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, box=False, box_color=array([0. , 0. , 0. , 0.6], dtype=float32), unit=None), text_overlay=TextOverlay(visible=False, color=array([0.5, 0.5, 0.5, 1. ], dtype=float32), font_size=10.0, position=<TextOverlayPosition.T